## HyperLang Sample Concept


In this sample concept, we will generate transcripts in Spanish given the country of origin and scenario. For now, we manually specify the country and scenario in this notebook, but, in the future we can create more
specific parameters (e.g. difficulty, number of speakers in conversation) that will be inputted by the end user.

### Helper Classes

In [1]:
"""
Module for handling text generation using Hugging Face's API and for providing
a teacher agent that can annotate or translate a given Spanish conversation.
"""

import os
import asyncio
import config
from typing import Any, AsyncGenerator
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv()


class PromptHandler:
    """
    Loads and formats prompt templates from a file.
    """

    def __init__(self, template_path: str) -> None:
        self.template_path = template_path

    def load_prompt(self) -> str:
        with open(self.template_path, 'r', encoding='utf-8') as file:
            return file.read()

    def format_prompt_scenario(self, scenario: str, country_name: str = "Spain") -> str:
        template = self.load_prompt()
        return template.format(
            scenario=scenario,
            country_name=country_name
        )

    def format_prompt(self, **kwargs) -> str:
        template = self.load_prompt()
        return template.format(**kwargs)


class TranscriptGenerator:
    """
    Generates or streams text from a Hugging Face model.
    """

    def __init__(self, model_name: str, temperature: float = 0.7) -> None:
        self.client = InferenceClient(api_key=os.getenv('HUGGINGFACE_TOKEN'))
        self.model_name = model_name
        self.temperature = temperature

    def generate_transcript(self, prompt: str) -> str:
        try:
            messages = [{"role": "user", "content": prompt}]
            completion = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=10000
            )
            return completion.choices[0].message.content
        except Exception as exc:
            return f"Error: {str(exc)}"

    async def generate_transcript_stream(
        self, prompt: str
    ) -> AsyncGenerator[str, None]:
        try:
            messages = [{"role": "user", "content": prompt}]
            stream = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=10000,
                stream=True
            )
            for chunk in stream:
                content = chunk.choices[0].delta.content
                if content:
                    yield content
                    await asyncio.sleep(0.01)
        except Exception as exc:
            yield f"Error: {str(exc)}"


class TeacherAgent:
    """
    Uses a teacher-specific prompt to provide annotated explanations or translations.
    """

    def __init__(
        self,
        template_path: str,
        model_name: str,
        temperature: float = 0.7
    ) -> None:
        self.prompt_handler = PromptHandler(template_path)
        self.transcript_generator = TranscriptGenerator(
            model_name=model_name,
            temperature=temperature
        )

    def explain_transcript(
        self,
        transcript: str,
        scenario: str,
        country_name: str = "Spain"
    ) -> str:
        teacher_prompt = self.prompt_handler.format_prompt(
            transcript=transcript,
            scenario=scenario,
            country_name=country_name
        )
        return self.transcript_generator.generate_transcript(teacher_prompt)

    async def explain_transcript_stream(
        self,
        transcript: str,
        scenario: str,
        country_name: str = "Spain"
    ) -> AsyncGenerator[str, None]:
        teacher_prompt = self.prompt_handler.format_prompt(
            transcript=transcript,
            scenario=scenario,
            country_name=country_name
        )
        async for chunk in self.transcript_generator.generate_transcript_stream(teacher_prompt):
            yield chunk

/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
country_name = "Colombia" ## specify the country (optional)

In [3]:
scenario_path = "./data/scenarios/cafe.txt"
with open(scenario_path, 'r') as file:
    scenario = file.read()
print(scenario)

A tourist enters a busy cafe and goes up to the counter to order.
They don't know exactly what to order and want some advice from the server.
After ordering and paying with card they take a seat and wait for their order to arrive.
Once it arrives they interact with the server and finsih their order and then leave.


In [4]:
print(config.PROMPT_TEMPLATE_PATH)
prompt_handler = PromptHandler(config.PROMPT_TEMPLATE_PATH)
prompt = prompt_handler.format_prompt_scenario(scenario, country_name=country_name)

./data/system_prompt_template.txt


In [5]:
print(prompt)

You are an expert content creator for Spanish language learners. Your role is to produce a natural-sounding, Spanish-only conversation between two characters. The conversation should be based on the following details:

- Country: Colombia
- Scenario: A tourist enters a busy cafe and goes up to the counter to order.
They don't know exactly what to order and want some advice from the server.
After ordering and paying with card they take a seat and wait for their order to arrive.
Once it arrives they interact with the server and finsih their order and then leave.

Requirements:

1. Spanish Only
   - Do not include any English text.
   - Reflect the local dialect and culture of Colombia in your Spanish.

2. Use Personal Names
   - Use realistic names (e.g., Ana, David, Camila) for each character depending on the context.
   - Avoid generic labels like “Visitor,” “Guide,” “Man,” or “Woman.”

3. No Background Text or Meta Commentary
   - Do not include narration, scene descriptions, or notes

## Synchronous Output

In [6]:
transcript_generator = TranscriptGenerator(model_name=config.MODEL_NAME)
initial_transcript = transcript_generator.generate_transcript(prompt)

## Asynchronous Output

In [7]:
async def stream_convo():
    async def convo_stream():
        async for token in transcript_generator.generate_transcript_stream(prompt):
            yield token
    async for token in convo_stream():
        print(token, end='', flush=True)

In [8]:
await stream_convo()

 **Café en Colombia**

Ana (el server): ¡Hola! ¿Qué podemos hacer para ti hoy?

David (el turista): Hola, Ana. ¿Me puedes recomendar algo? Soy un visitante y no sé mucho de la comida típica de Colombia.

Ana: ¡No hay problema! Pregunta lo que te guste comer y voy a asesorarte. ¿Has probado la café colombiana antes?

David: No lo he probado todavía. ¿Qué es lo mejor que probar?

Ana: La bandeja tipica es una opción excelente. Contiene arroz, carne, platano maduro, y una ensalada. También podrías probar la arepa con queso o la sopa de mondongo.

David: Ahi va, me gustaría probar la bandeja tipica, por favor.

Ana: ¡Perfecto! ¿Tendrás un tarjeta de crédito, o querrás pagar en efectivo?

David: Tendré una tarjeta de débito.

Ana: ¡Claro! Te espero a corto tiempo con tu pedido. ¡Gracias por elegir nuestro café!

David (se sienta y espera su pedido): ¡Muy bien! Estoy emocionado por probar la comida de Colombia.

Ana (entrega la comida): Aquí está tu pedido, ¡disfruta! Recuerda que si hay alg

## Adding a Teacher

In [9]:
teacher_agent = TeacherAgent(
    template_path=config.TEACHER_TEMPLATE_PATH,
    model_name=config.MODEL_NAME,
    temperature=0.2
)

In [10]:
teacher_response = teacher_agent.explain_transcript(transcript=initial_transcript,
    scenario=scenario,
    country_name=country_name)

In [11]:
print("=== Initial Transcript ===")
print(initial_transcript)
print("\n=== Teacher Explanation ===")
print(teacher_response)

=== Initial Transcript ===
Juan Pablo (el visitante): Hola, ¿podrías orientarme un poco sobre qué ordenar en este café?

Catalina (el servidor): ¡Claro! ¿Te gusta el café o quizás prefieres un refresco?

Juan Pablo: ¡Sí, claro! Me gusta el café, pero ¿qué es lo recomendado en este lugar?

Catalina: En este café, la especialidad es el Tinto con Pan de Bono. También tenemos refrescos y postres.

Juan Pablo: Ahorita no voy a ordenar nada más, pero me gustaría probar el Tinto con Pan de Bono.

Catalina: ¡Perfecto! Pagaré con tu tarjeta y te traeré la orden a tu mesa en unos minutos.

Juan Pablo: Gracias, Catalina.

(Se pasan unos minutos y la orden llega a la mesa de Juan Pablo)

Juan Pablo: ¡Muy bueno! Esto fue recomendado?

Catalina: Claro que sí, mi amor. Es una de las mejores comidas tradicionales de Colombia.

Juan Pablo: Espezialmente el Tinto y el Pan de Bono?

Catalina: Sí, exacto. ¡Espero que te guste! ¡Atrévete a probar otras cosas en tu próxima visita!

Juan Pablo: Gracias, Cata

In [12]:
async def stream_convo():
    async def convo_stream():
        async for token in teacher_agent.explain_transcript_stream(transcript=initial_transcript,
    scenario=scenario,
    country_name=country_name
):
            yield token
    async for token in convo_stream():
        print(token, end='', flush=True)

In [13]:
await stream_convo()

 -----
Juan Pablo (el visitante): Hola, ¿podrías orientarme un poco sobre qué ordenar en este café?

Maestro: Juan Pablo is asking the server for some guidance on what to order.

Catalina (el servidor): ¡Claro! ¿Te gusta el café o quizás prefieres un refresco?

Maestro: Catalina is asking if Juan Pablo prefers coffee or a drink.

Juan Pablo: ¡Sí, claro! Me gusta el café, pero ¿qué es lo recomendado en este lugar?

Maestro: Juan Pablo says he likes coffee, but asks what is recommended in this café.

Catalina: En este café, la especialidad es el Tinto con Pan de Bono. También tenemos refrescos y postres.

Maestro: Catalina tells Juan Pablo that the specialty of this café is Tinto con Pan de Bono, and they also have drinks and desserts.

Juan Pablo: Ahorita no voy a ordenar nada más, pero me gustaría probar el Tinto con Pan de Bono.

Maestro: Juan Pablo says he won't order anything else for now, but he'd like to try the Tinto con Pan de Bono.

Catalina: ¡Perfecto! Pagaré con tu tarjeta y 